In [1]:
import numpy as np
import pandas as pd

# pandas Time zone information

In [4]:
rng = pd.date_range('3/6/2016 00:00', periods = 15, freq = 'd')
print(rng.tz) # you don't care about time zone

None


In [3]:
rng

DatetimeIndex(['2016-03-06', '2016-03-07', '2016-03-08', '2016-03-09',
               '2016-03-10', '2016-03-11', '2016-03-12', '2016-03-13',
               '2016-03-14', '2016-03-15', '2016-03-16', '2016-03-17',
               '2016-03-18', '2016-03-19', '2016-03-20'],
              dtype='datetime64[ns]', freq='D')

In [5]:
rng = pd.date_range('3/6/2016 00:00', periods = 15, freq = 'd', tz = 'Europe/Paris')
rng.tz

<DstTzInfo 'Europe/Paris' LMT+0:09:00 STD>

### Getting lists of time zones

In [6]:
from pytz import common_timezones, all_timezones
print(len(common_timezones))

433


In [7]:
print(len(all_timezones))

596


### What are some time zones considered 'common'?

In [8]:
common_timezones[0:10]

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara',
 'Africa/Bamako',
 'Africa/Bangui',
 'Africa/Banjul',
 'Africa/Bissau',
 'Africa/Blantyre']

### Localizing a timestamp

In [9]:
t_naive = pd.Timestamp('2016-05-05 08:15')
t_naive

Timestamp('2016-05-05 08:15:00')

In [10]:
t = t_naive.tz_localize(tz = 'US/Central')
t

Timestamp('2016-05-05 08:15:00-0500', tz='US/Central')

In [11]:
t.tz_convert('Asia/Tokyo')

Timestamp('2016-05-05 22:15:00+0900', tz='Asia/Tokyo')

### What is the difference between tz_convert and tz_localize?

hint: try to run tz_convert on a naive time stamp

### Ambiguous times

In [12]:
# You'll get weirdness with timezones based on daylight savings:
rng = pd.date_range('2016-03-10', periods = 10, tz = 'US/Eastern')
ts = pd.Series(range(10), index = rng)
# What do you notice below?
ts

2016-03-10 00:00:00-05:00    0
2016-03-11 00:00:00-05:00    1
2016-03-12 00:00:00-05:00    2
2016-03-13 00:00:00-05:00    3
2016-03-14 00:00:00-04:00    4
2016-03-15 00:00:00-04:00    5
2016-03-16 00:00:00-04:00    6
2016-03-17 00:00:00-04:00    7
2016-03-18 00:00:00-04:00    8
2016-03-19 00:00:00-04:00    9
Freq: D, dtype: int64

In [16]:
# For the same reason you can run into 'ambiguous' dates
rng_hourly = pd.DatetimeIndex(['11/06/2011 00:00', '11/06/2011 01:00','11/06/2011 01:00', '11/06/2011 02:00','11/06/2011 03:00'])

In [17]:
rng_hourly

DatetimeIndex(['2011-11-06 00:00:00', '2011-11-06 01:00:00',
               '2011-11-06 01:00:00', '2011-11-06 02:00:00',
               '2011-11-06 03:00:00'],
              dtype='datetime64[ns]', freq=None)

In [18]:
# What happens if we localize?
rng_hourly.tz_localize('US/Central')

AmbiguousTimeError: Cannot infer dst time from 2011-11-06 01:00:00, try using the 'ambiguous' argument

### How do we deal with this ambiguous time error?
hint: http://pandas.pydata.org/pandas-docs/stable/timeseries.html#ambiguous-times-when-localizing

In [19]:
# %load snippets/ambig.py
rng_hourly.tz_localize('US/Central', ambiguous = 'infer')

DatetimeIndex(['2011-11-06 00:00:00-05:00', '2011-11-06 01:00:00-05:00',
               '2011-11-06 01:00:00-06:00', '2011-11-06 02:00:00-06:00',
               '2011-11-06 03:00:00-06:00'],
              dtype='datetime64[ns, US/Central]', freq=None)

### How can we check whether the inference did what we wanted?

In [20]:
# %load snippets/confirm.py
rng_hourly.tz_localize('US/Central', ambiguous = 'infer').tz_convert('utc')

DatetimeIndex(['2011-11-06 05:00:00+00:00', '2011-11-06 06:00:00+00:00',
               '2011-11-06 07:00:00+00:00', '2011-11-06 08:00:00+00:00',
               '2011-11-06 09:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

### Pandas goes to amazing length to try to figure things out for you

In [21]:
# What's going on here?
pd.Timestamp('2016-03-13 02:00', tz = 'US/Eastern') # time change

NonExistentTimeError: 2016-03-13 02:00:00